# Prepare Darknet-53 for classification using Darknet-53 trained using Imagenet (1000 classes).

##### 1.  Clone and install dependencies 

**IMPORTANT**: Restart following the instruction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!git clone https://github.com/qqwweee/keras-yolo3.git
%cd keras-yolo3/

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.07 KiB | 623.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/keras-yolo3


##### 2.  Check Tensorflow2 version

In [ ]:
!ls

import tensorflow as tf
tf.__version__

[Errno 2] No such file or directory: './keras-yolo3'
/content/keras-yolo3
coco_annotation.py  kmeans.py	train_bottleneck.py  yolo.py
convert.py	    LICENSE	train.py	     yolov3.cfg
darknet53.cfg	    model_data	voc_annotation.py    yolov3-tiny.cfg
font		    README.md	yolo3		     yolo_video.py


'2.2.0'

##### 3.  Convert Pretrained Darknet Weight (Not required )

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74 -O '/content/drive/My Drive/yolo-weights/darknet53.conv.74.weights'

--2020-06-10 16:02:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘/content/drive/My Drive/yolo-weights/darknet53.conv.74.weights’

/content/drive/My D 100%[===================>] 154.96M   142KB/s    in 14m 50s 

2020-06-10 16:17:41 (178 KB/s) - ‘/content/drive/My Drive/yolo-weights/darknet53.conv.74.weights’ saved [162482580/162482580]



In [ ]:
!python convert.py darknet53.cfg '/content/drive/My Drive/yolo-weights/darknet53.conv.74.weights' '/content/drive/My Drive/yolo-weights/darknet53_conv74_416px_imgnet.h5'

Using TensorFlow backend.
2020-06-10 16:20:32.693080: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading weights.
Weights Header:  0 2 0 [0]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
2020-06-10 16:20:34.578396: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-10 16:20:34.596129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-10 16:20:34.596946: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-06-10 16:20

In [ ]:
!python convert.py -h

Using TensorFlow backend.
2020-06-10 16:34:17.055511: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
usage: convert.py [-h] [-p] [-w] config_path weights_path output_path

Darknet To Keras Converter.

positional arguments:
  config_path         Path to Darknet cfg file.
  weights_path        Path to Darknet weights file.
  output_path         Path to output Keras model file.

optional arguments:
  -h, --help          show this help message and exit
  -p, --plot_model    Plot generated Keras model and save as image.
  -w, --weights_only  Save as Keras weights file instead of model file.


In [ ]:
#!python convert.py -p -w darknet53.cfg '/content/drive/My Drive/yolo-weights/darknet53.conv.74.weights' '/content/drive/My Drive/yolo-weights/darknet53.conv.74.414px.weights_only.h5'
!python convert.py -p darknet53.cfg '/content/drive/My Drive/yolo-weights/darknet53.conv.74.weights' '/content/drive/My Drive/yolo-weights/darknet53.conv.74.414px.weights_n_model.h5'

Using TensorFlow backend.
2020-06-10 18:46:54.174291: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading weights.
Weights Header:  0 2 0 [0]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
2020-06-10 18:46:56.251119: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-10 18:46:56.307203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-10 18:46:56.307810: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-

Validate the H5 file ( model + weights )

In [ ]:
from numpy import loadtxt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications.imagenet_utils import (preprocess_input, decode_predictions)
 
# load model
darknet53_model = load_model('/content/drive/My Drive/yolo-weights/darknet53_conv74_416px_imgnet.h5')

# mark loaded layers as not trainable
for layer in darknet53_model.layers:
	layer.trainable = False

# summarize model.
add_layer = tf.keras.layers.GlobalAveragePooling2D()((darknet53_model.layers[-1].output))

# Add new Global Average pooling layer and save the model for experiments
model = Model(inputs=darknet53_model.inputs, outputs=add_layer)

# Compile using the same specs as Darknet-53.cfg
optimizer = tf.keras.optimizers.SGD(lr=0.001, decay=0.0005, momentum=0.9)
model.compile(loss='categorical_crossentropy',
                    optimizer=optimizer,
                    metrics=['accuracy'])

plot_model(model, to_file='/content/drive/My Drive/yolo-weights/dark53_416px_imgnet_final.png')
model.summary()
model.save(filepath="/content/drive/My Drive/yolo-weights/darknet53_416px_imgnet_final.tf")

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

Example to use the bottle neck CONV model...

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential

# Load model

model_dark53 = load_model(filepath="/content/drive/My Drive/yolo-weights/darknet53_416px_imgnet_final.tf", compile=True)

# Check the architecture...
#print(model_dark53.outputs)

dense_model = Sequential([model_dark53,
                          tf.keras.layers.Flatten(),
                          Dense(512, activation='softmax'),
                          Dense(3, activation='softmax')]
                         )

#x = tf.keras.layers.Flatten()((model_dark53.outputs)[0])
#dense_output = Dense(512, activation='softmax')(x)
#dense_output = Dense(3, activation='softmax')(x)

# define new model
#dense_model = Model(inputs=darknet53_model.inputs, outputs=dense_output)

# Compile using a learning rate less than specified in darknet53.cfg
optimizer = Adam(learning_rate=0.0001)
dense_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizer,
                    metrics=['accuracy'])

dense_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Model)              (None, 1024)              40620640  
_________________________________________________________________
flatten_11 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 1539      
Total params: 41,146,979
Trainable params: 41,111,267
Non-trainable params: 35,712
_________________________________________________________________


Predict an image using the Darknet-53 imagenet classification model...

In [ ]:
# load an image from file
image = load_img('/content/drive/My Drive/yolo-weights/mug.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# predict the probability across all output classes
yhat = darknet53_model.predict(image)
print("Shape of output :",yhat.shape)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

In [ ]:
!ls -l '/content/drive/My Drive/yolo-weights/mug.jpg'

-rw------- 1 root root 172498 Jun 10 19:10 '/content/drive/My Drive/yolo-weights/mug.jpg'


................New Notebook validated only until here.............